# Imports

In [19]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import pathlib
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from tqdm import tqdm

# Data paths

In [20]:
#DATA_ROOT = pathlib.Path(r"data\archive\food-101\food-101")
#IMAGE_DIR = pathlib.Path("data/archive/food-101/food-101/images")

DATA_ROOT = pathlib.Path(r"E:\\food-101\\food-101")
IMAGE_DIR = pathlib.Path("E:\\food-101\\food-101\\images")

if not IMAGE_DIR.exists():
    raise FileNotFoundError(f"Missing dataset folder: {IMAGE_DIR}")

print("Dataset path OK.")


Dataset path OK.


# Data preprocessing

In [21]:
img_size = 224

train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

full_dataset = datasets.ImageFolder(IMAGE_DIR, transform=train_transform, allow_empty=True)
class_names = full_dataset.classes
num_classes = len(class_names)

print("Total images:", len(full_dataset))
print("Classes:", num_classes)


Total images: 101000
Classes: 101


# Split data into sets

In [22]:
train_size = int(0.7 * len(full_dataset))
val_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(full_dataset, [train_size, val_size, test_size])

# Create Dataloaders

In [23]:
val_ds.dataset.transform = test_transform
test_ds.dataset.transform = test_transform

batch_size = 32

train_loader = DataLoader(
    train_ds,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)


# Create Model

In [24]:
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
model = efficientnet_b0(weights=weights)

# Replace classifier head
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Freeze feature extractor
for param in model.features.parameters():
    param.requires_grad = False

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


Device: cuda


# Define training and evaluation functions

In [25]:

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for imgs, labels in tqdm(loader, desc="Training", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    return total_loss / total, correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

    return total_loss / total, correct / total


# Train model

In [26]:
epochs = 5

for epoch in range(epochs):
    # ---- Debug: hardware info ----
    print(f"\n--- Epoch {epoch+1}/{epochs} Diagnostics ---")
    print("Using device:", device)

    # Check model parameters
    first_param = next(model.parameters())
    print("Model param device:", first_param.device)

    if device == "cuda":
        print("Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
        print("Reserved:", torch.cuda.memory_reserved() / 1024**2, "MB")

    print("-------------------------------------------\n")

    # ---- Training ----
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)

    # ---- Validation ----
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    # ---- Epoch Summary ----
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")


--- Epoch 1/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 87.54541015625 MB
Reserved: 662.0 MB
-------------------------------------------



Epoch 1/5
  Train Loss: 2.5606, Acc: 0.4107
  Val   Loss: 2.0274, Acc: 0.5018

--- Epoch 2/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 89.0263671875 MB
Reserved: 668.0 MB
-------------------------------------------



Epoch 2/5
  Train Loss: 2.0194, Acc: 0.5044
  Val   Loss: 1.9290, Acc: 0.5244

--- Epoch 3/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 89.0263671875 MB
Reserved: 668.0 MB
-------------------------------------------



Epoch 3/5
  Train Loss: 1.9219, Acc: 0.5213
  Val   Loss: 1.9018, Acc: 0.5263

--- Epoch 4/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 89.0263671875 MB
Reserved: 668.0 MB
-------------------------------------------



Epoch 4/5
  Train Loss: 1.8626, Acc: 0.5354
  Val   Loss: 1.8853, Acc: 0.5320

--- Epoch 5/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 89.0263671875 MB
Reserved: 668.0 MB
-------------------------------------------



Epoch 5/5
  Train Loss: 1.8307, Acc: 0.5406
  Val   Loss: 1.8868, Acc: 0.5330


# Test model

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Evaluating", leave=False)
        
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            pbar.set_postfix({
                "loss": f"{loss.item():.4f}",
                "acc": f"{100 * correct/total:.2f}%"
            })

    avg_loss = running_loss / total
    avg_acc = correct / total

    return avg_loss, avg_acc

In [29]:
test_loss, test_acc = evaluate(model, test_loader, criterion, device)

print("\n=== Test Results ===")
print(f"Test Loss:     {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print("====================\n")


=== Test Results ===
Test Loss:     1.8994
Test Accuracy: 0.5361

